In [35]:
import pandas as pd
from pandas import json_normalize
from sqlalchemy import create_engine
import requests
import json
import time
import psycopg2
import os

### Extract CSVs into DataFrames

In [36]:
hospital_data = "Resources/Medicare_Inpatient_Hospital_by_Provider_and_Service_2019.csv"

In [37]:
hospital_df = pd.read_csv(hospital_data)
hospital_df.head()

,Rndrng_Prvdr_CCN,Rndrng_Prvdr_Org_Name,Rndrng_Prvdr_St,Rndrng_Prvdr_City,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,DRG_Cd,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",14,"$326,514.86","$62,788.07","$61,050.29"
1,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,23,CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE COM...,55,"$140,874.53","$29,766.84","$27,205.18"
2,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,24,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,20,"$109,788.10","$22,780.30","$20,067.75"
3,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,25,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,23,"$124,579.26","$24,107.30","$22,764.74"
4,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,27,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,16,"$75,029.06","$18,216.38","$10,206.88"


In [38]:
# Rename the column headers
hospital_df = hospital_df.rename(columns={"Rndrng_Prvdr_CCN": "provider_CCN",
                                                          "Rndrng_Prvdr_Org_Name": "org_name",
                                                          "Rndrng_Prvdr_St": "street_address", 
                                         "Rndrng_Prvdr_City": "city", 
                                         "Rndrng_Prvdr_State_Abrvtn": "state_abv", 
                                         "Rndrng_Prvdr_State_FIPS": "state_FIPS",
                                        "Rndrng_Prvdr_Zip5": "zip_code", 
                                         "Rndrng_Prvdr_RUCA" : "RUCA", 
                                         "Rndrng_Prvdr_RUCA_Desc" : "RUCA_desc", 
                                         "DRG_Cd" : "DRG_code", 
                                         "DRG_Desc" : "DRG_desc", 
                                         "Tot_Dschrgs" : "total_discharges", 
                                         "Avg_Submtd_Cvrd_Chrg" : "avg_submitted_covered_charges", 
                                         "Avg_Tot_Pymt_Amt" : "avg_total_payment_amnt", 
                                         "Avg_Mdcr_Pymt_Amt": "avg_medicare_payment_amnt"})


In [39]:
#display dataframe 
hospital_df.head()

,provider_CCN,org_name,street_address,city,state_abv,state_FIPS,zip_code,RUCA,RUCA_desc,DRG_code,DRG_desc,total_discharges,avg_submitted_covered_charges,avg_total_payment_amnt,avg_medicare_payment_amnt
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",14,"$326,514.86","$62,788.07","$61,050.29"
1,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,23,CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE COM...,55,"$140,874.53","$29,766.84","$27,205.18"
2,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,24,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,20,"$109,788.10","$22,780.30","$20,067.75"
3,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,25,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,23,"$124,579.26","$24,107.30","$22,764.74"
4,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,27,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,16,"$75,029.06","$18,216.38","$10,206.88"


In [40]:
#get data from centers for medicare and medicaid services API and save to response variable 
#base_url = "https://data.cms.gov/data-api/v1/dataset/5fccd951-9538-48a7-9075-6f02b9867868/data?column=Rndrng_NPI&column=Rndrng_Prvdr_Last_Org_Name&column=Rndrng_Prvdr_First_Name&column=Rndrng_Prvdr_St1&column=Rndrng_Prvdr_Type&column=Rndrng_Prvdr_State_Abrvtn&offset=1000&size=10000&limit=10000"
base_url = "https://data.cms.gov/data-api/v1/dataset/5fccd951-9538-48a7-9075-6f02b9867868/data"
response = requests.get(base_url).json()

#verify that the call actually pulled the data 
print(response[0])

{'Rndrng_NPI': '1003000126', 'Rndrng_Prvdr_Last_Org_Name': 'Enkeshafi', 'Rndrng_Prvdr_First_Name': 'Ardalan', 'Rndrng_Prvdr_MI': '', 'Rndrng_Prvdr_Crdntls': 'M.D.', 'Rndrng_Prvdr_Gndr': 'M', 'Rndrng_Prvdr_Ent_Cd': 'I', 'Rndrng_Prvdr_St1': '900 Seton Dr', 'Rndrng_Prvdr_St2': '', 'Rndrng_Prvdr_City': 'Cumberland', 'Rndrng_Prvdr_State_Abrvtn': 'MD', 'Rndrng_Prvdr_State_FIPS': '24', 'Rndrng_Prvdr_Zip5': '21502', 'Rndrng_Prvdr_RUCA': '1', 'Rndrng_Prvdr_RUCA_Desc': 'Metropolitan area core: primary flow within an urbanized area of 50,000 and greater', 'Rndrng_Prvdr_Cntry': 'US', 'Rndrng_Prvdr_Type': 'Internal Medicine', 'Rndrng_Prvdr_Mdcr_Prtcptg_Ind': 'Y', 'HCPCS_Cd': '99217', 'HCPCS_Desc': 'Hospital observation care on day of discharge', 'HCPCS_Drug_Ind': 'N', 'Place_Of_Srvc': 'F', 'Tot_Benes': '40', 'Tot_Srvcs': '40', 'Tot_Bene_Day_Srvcs': '40', 'Avg_Sbmtd_Chrg': '232.275', 'Avg_Mdcr_Alowd_Amt': '72.59', 'Avg_Mdcr_Pymt_Amt': '57.87', 'Avg_Mdcr_Stdzd_Amt': '58.19175'}


In [41]:
#initialize empty lists to store parts of the response 
Rndrng_NPI = []
Rndrng_Prvdr_Last_Org_Name = []
Rndrng_Prvdr_First_Name = []
Rndrng_Prvdr_St1 = []
Rndrng_Prvdr_Type = []
Rndrng_Prvdr_State_Abrvtn = []

In [42]:
#initialize empty lists to store the data from the API pull to eventually put into dataframe 
Rndrng_NPI = []
Rndrng_Prvdr_Last_Org_Name=[]
Rndrng_Prvdr_First_Name=[]
Rndrng_Prvdr_St1=[]
Rndrng_Prvdr_Type=[]
Rndrng_Prvdr_State_Abrvtn = []

In [43]:
#loop through JSON response and add data to the appropriate lists 

for x in range(len(response)): 
    try: 
        Rndrng_NPI.append(response[x]["Rndrng_NPI"])
        Rndrng_Prvdr_Last_Org_Name.append(response[x]["Rndrng_Prvdr_Last_Org_Name"])
        Rndrng_Prvdr_First_Name.append(response[x]["Rndrng_Prvdr_First_Name"])
        Rndrng_Prvdr_St1.append(response[x]["Rndrng_Prvdr_St1"])
        Rndrng_Prvdr_Type.append(response[x]["Rndrng_Prvdr_Type"])
        Rndrng_Prvdr_State_Abrvtn.append(response[x]["Rndrng_Prvdr_State_Abrvtn"])
    except: 
        print(f"There is an issue with the data row {x}. Skipping.")

In [44]:
#verify that the arrays are the same length in order to create dataframe 
print(len(Rndrng_NPI), len(Rndrng_Prvdr_Last_Org_Name), len(Rndrng_Prvdr_First_Name), len(Rndrng_Prvdr_St1), len(Rndrng_Prvdr_Type), len(Rndrng_Prvdr_State_Abrvtn))

1000 1000 1000 1000 1000 1000


In [45]:
#create dataframe based on the lists 
practitioner_df = pd.DataFrame({"NPI": Rndrng_NPI, "last_name" : Rndrng_Prvdr_Last_Org_Name,
                                "first_name": Rndrng_Prvdr_First_Name, "street_address":Rndrng_Prvdr_St1, 
                               "provider_type" : Rndrng_Prvdr_Type, "state_abv" : Rndrng_Prvdr_State_Abrvtn})
practitioner_df

,NPI,last_name,first_name,street_address,provider_type,state_abv
0,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
1,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
2,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
3,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
4,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
...,...,...,...,...,...,...
995,1003007170,Danciu,Sorin,3118 N Ashland Ave,Cardiology,IL
996,1003007204,Yamani,Hussein,100 Medical Center Blvd,Cardiology,TX
997,1003007204,Yamani,Hussein,100 Medical Center Blvd,Cardiology,TX
998,1003007204,Yamani,Hussein,100 Medical Center Blvd,Cardiology,TX


In [46]:
# join the tables 
joined_df = hospital_df.join(practitioner_df.set_index('street_address'), on='street_address', how = 'outer', lsuffix = 'H', rsuffix = 'P')

#drop the rows where we don't have practitioner data 
joined_df = joined_df.dropna()

#drop duplicate columns
joined_df = joined_df.drop(columns = ['state_abvH'])

#display df 
joined_df

,provider_CCN,org_name,street_address,city,state_FIPS,zip_code,RUCA,RUCA_desc,DRG_code,DRG_desc,total_discharges,avg_submitted_covered_charges,avg_total_payment_amnt,avg_medicare_payment_amnt,NPI,last_name,first_name,provider_type,state_abvP
25564.0,60024.0,University Of Colorado Hospital Authority,12605 E 16th Ave,Aurora,8.0,80045.0,1.0,Metropolitan area core: primary flow within an...,1.0,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,20,"$1,032,386.50","$213,169.45","$202,130.45",1003000480,Rothchild,Kevin,General Surgery,CO
25564.0,60024.0,University Of Colorado Hospital Authority,12605 E 16th Ave,Aurora,8.0,80045.0,1.0,Metropolitan area core: primary flow within an...,1.0,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,20,"$1,032,386.50","$213,169.45","$202,130.45",1003000480,Rothchild,Kevin,General Surgery,CO
25564.0,60024.0,University Of Colorado Hospital Authority,12605 E 16th Ave,Aurora,8.0,80045.0,1.0,Metropolitan area core: primary flow within an...,1.0,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,20,"$1,032,386.50","$213,169.45","$202,130.45",1003000480,Rothchild,Kevin,General Surgery,CO
25565.0,60024.0,University Of Colorado Hospital Authority,12605 E 16th Ave,Aurora,8.0,80045.0,1.0,Metropolitan area core: primary flow within an...,3.0,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",55,"$1,295,619.27","$200,194.76","$185,340.09",1003000480,Rothchild,Kevin,General Surgery,CO
25565.0,60024.0,University Of Colorado Hospital Authority,12605 E 16th Ave,Aurora,8.0,80045.0,1.0,Metropolitan area core: primary flow within an...,3.0,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",55,"$1,295,619.27","$200,194.76","$185,340.09",1003000480,Rothchild,Kevin,General Surgery,CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122341.0,330285.0,Strong Memorial Hospital,601 Elmwood Ave,Rochester,36.0,14642.0,1.0,Metropolitan area core: primary flow within an...,981.0,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,12,"$115,204.83","$45,464.83","$37,149.17",1003002643,Bradt,Carie,Physician Assistant,NY
122341.0,330285.0,Strong Memorial Hospital,601 Elmwood Ave,Rochester,36.0,14642.0,1.0,Metropolitan area core: primary flow within an...,981.0,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,12,"$115,204.83","$45,464.83","$37,149.17",1003002643,Bradt,Carie,Physician Assistant,NY
122341.0,330285.0,Strong Memorial Hospital,601 Elmwood Ave,Rochester,36.0,14642.0,1.0,Metropolitan area core: primary flow within an...,981.0,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,12,"$115,204.83","$45,464.83","$37,149.17",1003002643,Bradt,Carie,Physician Assistant,NY
122341.0,330285.0,Strong Memorial Hospital,601 Elmwood Ave,Rochester,36.0,14642.0,1.0,Metropolitan area core: primary flow within an...,981.0,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,12,"$115,204.83","$45,464.83","$37,149.17",1003002643,Bradt,Carie,Physician Assistant,NY


### Transform Data


In [47]:
joined_df['avg_submitted_covered_charges'] = joined_df['avg_submitted_covered_charges'].str.replace(',', '')
joined_df['avg_total_payment_amnt'] = joined_df['avg_total_payment_amnt'].str.replace(',', '')
joined_df['avg_medicare_payment_amnt'] = joined_df['avg_medicare_payment_amnt'].str.replace(',', '')
joined_df['total_discharges'] = joined_df['total_discharges'].str.replace(',', '')
joined_df['avg_submitted_covered_charges'] = joined_df['avg_submitted_covered_charges'].str.replace("'", '')
joined_df['avg_total_payment_amnt'] = joined_df['avg_total_payment_amnt'].str.replace("'", '')
joined_df['avg_medicare_payment_amnt'] = joined_df['avg_medicare_payment_amnt'].str.replace("'", '')

### Create database connection

In [48]:
connection_string = "postgres:postgres@localhost:5432/healthcost_db"
engine = create_engine(f'postgresql://{connection_string}')

In [49]:
# Confirm tables
engine.table_names()

/var/folders/n7/sntrh25x5b158s65t0zfwdv40000gn/T/ipykernel_1850/3015872754.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['joined_df', 'practitioner', 'hospital']

### Load DataFrame into database

In [50]:
joined_df.to_csv(os.path.join('Output_CSVs/joined_df.csv'), index=False)
practitioner_df.to_csv(os.path.join('Output_CSVs/practitioner.csv'), index=False)
hospital_df.to_csv(os.path.join('Output_CSVs/hospital.csv'), index=False)


In [34]:
joined_df.to_sql(name='joined_df', con=engine, if_exists='append', index=True)
practitioner_df.to_sql(name='practitioner', con=engine, if_exists='append', index=True)
hospital_df.to_sql(name='hospital', con=engine, if_exists='append', index=True)